In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


2024-03-30 13:54:15.312470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-30 13:54:15.312502: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataset_dir = './data/'  # Update this to the path where your dataset is located
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'test')

# Parameters
batch_size = 16
img_height = 500
img_width = 500


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2342 images belonging to 28 classes.
Found 236 images belonging to 27 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


2024-03-30 13:54:22.665890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-30 13:54:22.666079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-30 13:54:22.666138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-03-30 13:54:22.666173: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-03-30 13:54:22.666219: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)


Epoch 1/20


2024-03-30 13:54:24.556804: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 943718400 exceeds 10% of free system memory.
2024-03-30 13:54:24.614837: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 943718400 exceeds 10% of free system memory.


146/146 [==============================] - 334s 2s/step - loss: -497222909952.0000 - accuracy: 0.0353 - val_loss: -4278163079168.0000 - val_accuracy: 0.0402
Epoch 2/20
146/146 [==============================] - 337s 2s/step - loss: -87567370289152.0000 - accuracy: 0.0353 - val_loss: -344151736451072.0000 - val_accuracy: 0.0357
Epoch 3/20
146/146 [==============================] - 316s 2s/step - loss: -1375550920720384.0000 - accuracy: 0.0353 - val_loss: -3707380873297920.0000 - val_accuracy: 0.0357
Epoch 4/20
146/146 [==============================] - 287s 2s/step - loss: -8584154102890496.0000 - accuracy: 0.0353 - val_loss: -17994000636051456.0000 - val_accuracy: 0.0402
Epoch 5/20
146/146 [==============================] - 284s 2s/step - loss: -32014662602063872.0000 - accuracy: 0.0353 - val_loss: -57304415735906304.0000 - val_accuracy: 0.0402
Epoch 6/20
 66/146 [============>.................] - ETA: 3:08 - loss: -67161043557679104.0000 - accuracy: 0.0249

: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
